In [1]:
elastic_rods_dir = '../../elastic_rods/python'
weaving_dir = '../'
import os.path as osp
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
import numpy.linalg as la
from bending_validation import suppress_stdout as so
import matplotlib.pyplot as plt
import copy

In [2]:
# Reloadable libraries
import analysis_helper, mesh_vis, linkage_utils, compute_curve_from_curvature, importlib
importlib.reload(analysis_helper)
importlib.reload(mesh_vis)
importlib.reload(linkage_utils)
importlib.reload(compute_curve_from_curvature)
from analysis_helper import (compare_turning_angle,
                            is_on_sphere, 
                            get_distance_to_center_scalar_field, 
                            plot_curvatures, 
                            construct_elastic_rod_loop_from_rod_segments, 
                            concatenate_rod_properties_from_rod_segments, 
                            compute_min_distance_rigid_transformation)
from compute_curve_from_curvature import (match_geo_curvature_and_edge_len,
                                          get_curve_from_angle,
                                          get_laser_cutting_patter)
from ribbon_linkage_helper import export_linkage_geometry_to_obj
from linkage_utils import order_segments_by_strips, get_turning_angle_and_length_from_ordered_rods

In [3]:
# Sphere Parameters
default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
RIBBON_CS = [1, 15]
ISO_CS = [4, 4]
MODEL_PATH = osp.join(weaving_dir + 'models/equal_edge_atI.obj')
SUBDIVISION_RESOLUTION = 20

In [4]:
# Pseudosphere
default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
RIBBON_CS = [0.001, 0.01]
ISO_CS = [0.01, 0.01]
MODEL_NAME = "pseudo_sphere"
MODEL_PATH = osp.join(weaving_dir + 'models/{}.obj'.format(MODEL_NAME))
SUBDIVISION_RESOLUTION = 10
REGULARIZATION_WEIGHT = 0.01

In [13]:
def initialize_linkage(cross_section = ISO_CS, subdivision_res = SUBDIVISION_RESOLUTION, model_path = MODEL_PATH, cam_param = default_camera_parameters, regularization_weight = 0.1):
    l = elastic_rods.RodLinkage(model_path, subdivision_res, False)
    driver=l.centralJoint()
    l.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, cross_section, stiffAxis=elastic_rods.StiffAxis.D1))
    jdo = l.dofOffsetForJoint(driver)
    fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
    elastic_rods.compute_equilibrium(l, fixedVars=fixedVars)
    view = linkage_vis.LinkageViewer(l, width=1024, height=640)
    view.setCameraParams(cam_param)
    elastic_rods.restlen_solve(l)
    view.update(preserveExisting=True)
    elastic_rods.compute_equilibrium(l, fixedVars=fixedVars)
    return l, view

In [14]:
with so(): linkage, view = initialize_linkage(model_path = MODEL_PATH, cam_param = default_camera_parameters, cross_section = RIBBON_CS, subdivision_res = SUBDIVISION_RESOLUTION, regularization_weight = REGULARIZATION_WEIGHT)
export_linkage_geometry_to_obj(linkage, '{}_straight_linkage.obj'.format(MODEL_NAME))
view.show()

Renderer(camera=PerspectiveCamera(aspect=1.6, children=(PointLight(color='white', intensity=0.6, position=(0.0…

In [15]:
linkage.gradient()

array([ 1.67081739e-12,  7.16673832e-12,  1.14735644e-11, ...,
       -1.11769843e-14, -1.01807353e-10, -5.97675888e-12])